# Detect the shuttle discontinuity

To detect the inconsistency caused by the shuttle mode and clip the image and segmentation at that step.

Pipeline:

 - take z-axis of the image and clip in half-lower and half-upper
 - would it be easy to interpolate the image instead of array extracted from segmentation? if not, what is the best pipeline for having the upper and lower parts of the vessel segmentation connected and consistant?

## Define a clipping Plane

In [27]:
import os
import vtk
import numpy as np
from utilities import ReadVTKFile, WriteVTKFile, WriteVTUFile, ReadVTUFile


test_image = "/Users/ana/Documents/AnahitaSeresti/06_ContrastDispersion/1_CTPImages/VA02B/CTP/VA02B_06.vtk"

In [3]:
image = ReadVTKFile(test_image)
image

<vtkmodules.vtkCommonDataModel.vtkStructuredPoints(0x7fc3727f9c50) at 0x1a45a0ee0>

In [4]:
image.GetBounds()

(-66.8046875, 132.8046875, -257.3046875, -57.6953125, -288.5, -185.5)

In [5]:
x_min = image.GetBounds()[0]
x_max = image.GetBounds()[1]

y_min = image.GetBounds()[2]
y_max = image.GetBounds()[3]

z_min = image.GetBounds()[4]
z_max = image.GetBounds()[5]

center_point = [
    (x_max + x_min)/2,
    (y_max + y_min)/2,
    (z_max + z_min)/2
]
center_point

[33.0, -157.5, -237.0]

In [25]:
Plane = vtk.vtkPlane()
Plane.SetOrigin(center_point) # takes the centre of the image as the origin of the plane
Plane.SetNormal([0,0,1]) # takes the z-axis as the normal of the plane since the shuttle mode is in z-axis


def ImageClipper(image:vtk.vtkImageData, Plane:vtk.vtkPlane, InsideOut:bool=False):
    Clipper = vtk.vtkClipVolume()
    Clipper.AddInputData(image)
    Clipper.SetClipFunction(Plane)
    Clipper.SetInsideOut(InsideOut)
    Clipper.Update()

    return Clipper.GetOutput()



In [23]:
opath = os.path.splitext(test_image)[0] + "_upper.vtu"
WriteVTUFile(opath, ImageClipper(image, Plane, False))

In [24]:
opath = os.path.splitext(test_image)[0] + "_lower.vtu"
WriteVTUFile(opath, ImageClipper(image, Plane, True))

Pipeline:
 - define a clipper to clip the CTP images to upper and lowerimages.
 - use the same plane to clip the vessel segmentation into upper and lower parts.

        How to detect if a lumen only exists in either upper or lower portion??????
 - Create a dictionary of average attenuation values along the lumen.

In [ ]:
# to clip the lumen
def GridClipper(grid:vtk.vtkUnstructuredGrid, Plane:vtk.vtkPlane, InsideOut:bool = False):
    Clipper = vtk.vtkClipDataSet()
    Clipper.AddInputData(grid)
    Clipper.SetClipFunction(Plane)
    Clipper.SetInsideOut(InsideOut)
    Clipper.Update()

    return (Clipper.GetOutput())

In [30]:
lumen_path = "/Users/ana/Documents/AnahitaSeresti/06_ContrastDispersion/3_VesselProjection/VA02B/SVG_LAD/SVG_LAD.vtu"
Lumen = ReadVTUFile(lumen_path)

In [31]:
# clipping the lumen to upper and lower parts
Lumen_upper = GridClipper(Lumen, Plane, False)
Lumen_lower = GridClipper(Lumen, Plane, True)

In [34]:
Lumen_lower.GetNumberOfPoints() # use this to find if the lumen exists in both of the upper and lower portions.

0

todo:

write a script that does the preprocessing to compensate for the shuttle mode including the following pipeline:

 - loop over all of the images and split them to upper and lower portions.
 - take the lumen and split it into the upper and lower segments.
 - check if the lumen exists in both of the upper and lower portions of the image.
 - project the upper image on the upper segment and vice versa.
 - extract the cross sectional average along the centreline.
 - Interpolate with a factor of 2 in time domain, and re-order the attenuation array along the vessel.
 - Create an attenuation matrix of the washout phase.
 - Interpolate the attenuation matrix in x and t domains.
 - Export the attenuation matrix as the output of the script.

still one single problem exists: how to detect the upslope, peak and washout phases on the TAC?
Does the upper portion always lag the lower portion in imaging?